In [18]:
#import
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import scipy.cluster.hierarchy as hr
from geojson import LineString
from geojson import MultiLineString
from shapely.geometry import MultiLineString
import random

from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import pairwise_distances
import pandas as pd

import folium
from shapely import wkt, geometry
import json
from pprint import pprint
from openrouteservice import client, places

api_key = '5b3ce3597851110001cf62487a4f3ad33ea14791b72150c9e6446e5e'
clnt = client.Client(key=api_key)

In [19]:
#GPS data open and read
a=[]
with open('C:/Users/찬란한 돌멩이/Desktop/GPS DAT/Test DAT/GPS.txt', 'r') as f:
    lines = f.readlines()
#GPS to list
	
    for i in lines:
        b= i.split(sep=',')
        b[6]=b[6].replace("\n","")
        b[6]=b[6].replace(":","")
        a.append(b)

#Pre-processing(위도, 경도, 시간 값만 추출)
n_a = np.array(a)
for i in range(4):
    n_a = np.delete(n_a,2,axis=1)
n_a = n_a.astype(np.float64)
GPS_info_list1 = n_a.tolist()

In [20]:
#DTW 분석을 위한 Dummy 생성
N = len(GPS_info_list1)//100+1
for i in range(N):
    if i*100+100 > len(GPS_info_list1):
        globals()['GPS_dummy_'+str(i)] = GPS_info_list1[i*100:len(GPS_info_list1)+1]
    else:
        globals()['GPS_dummy_'+str(i)] = GPS_info_list1[i*100:i*100+100]


distance_matrix_dtw=np.zeros(shape=(N,N))
path_list=[]

#DTW분석
for i in range(len(GPS_info_list1)//100+1):
    for j in range(i+1, len(GPS_info_list1)//100+1):
        distance, path = fastdtw(globals()['GPS_dummy_'+str(i)],globals()['GPS_dummy_'+str(j)] , dist=euclidean)
        distance_matrix_dtw[i][j] = distance
        path_list.append(path) 
distance_matrix_dtw = distance_matrix_dtw.tolist()

In [21]:
#Agglomerative clustering
n_clusters=12
agg = AgglomerativeClustering(n_clusters= n_clusters, affinity='precomputed',
                              linkage='average')
u = agg.fit_predict(distance_matrix_dtw)

c:\anaconda3\envs\project23\lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [22]:
#loading Agglomerative Clustering result on differnt lists(클러스터 수만큼의 리스트들)

a=0
b=0
for i in range(n_clusters):
    globals()['Clusters_'+str(i)]=[]
    temp=[]
    
    for j in u:
        if j == a:
            temp.append(b)
        b+=1
    b=0
    globals()['Clusters_'+str(i)] = temp
    a+=1

In [23]:
#initializing_1
n_of_GPS_dummies =len(GPS_info_list1)//100+1
for i in range(n_of_GPS_dummies):
    globals()['temp_'+str(i)]=globals()['GPS_dummy_'+str(i)]

In [24]:
#initializing_2
n_of_GPS_dummies =len(GPS_info_list1)//100+1
for i in range(n_of_GPS_dummies):
    globals()['GPS_dummy_'+str(i)]=globals()['temp_'+str(i)]

#based on Clustering results, Making Clustered GPS path lists.
for i in range(n_clusters):
    globals()['Clusters_path_'+str(i)]=[]
    for j in globals()['Clusters_'+str(i)]:
        globals()['GPS_dummy_'+str(j)] = np.array(globals()['GPS_dummy_'+str(j)])
        globals()['GPS_dummy_'+str(j)] = np.delete(globals()['GPS_dummy_'+str(j)],2,1)
        globals()['GPS_dummy_'+str(j)][:,[0,1]] = globals()['GPS_dummy_'+str(j)][:,[1,0]] 
        globals()['Clusters_path_'+str(i)].append(globals()['GPS_dummy_'+str(j)])


In [25]:
#making path polygons[ith_cluster][jth_path]
for i in range(n_clusters):
    for j in range(len(globals()['Clusters_path_'+str(i)])):
        globals()['path_polygon_'+str(i)+str(j)]=[]
        for k in globals()['Clusters_path_'+str(i)][j]:
            globals()['path_polygon_'+str(i)+str(j)].append(str(k[0])+" "+str(k[1])+" ")
        globals()['path_polygon_'+str(i)+str(j)] = ','.join(globals()['path_polygon_'+str(i)+str(j)])
        globals()['path_polygon_'+str(i)+str(j)]= 'LineString'+" "+"("+ globals()['path_polygon_'+str(i)+str(j)] +")"


In [29]:
a=random.randint(0,n_clusters-1)

wkt_str = globals()['path_polygon_'+str(a)+str(len(globals()['Clusters_path_'+str(a)])-1)]
aoi_geom = wkt.loads(wkt_str)

aoi_coords = list(aoi_geom.coords)
aoi_coords = [(y,x) for x,y in aoi_coords] 
aoi_centroid = aoi_geom.centroid 


m = folium.Map(location=(aoi_centroid.y, aoi_centroid.x), zoom_start=16)

for i in range(0,len(globals()['Clusters_path_'+str(a)])):
    globals()['wkt_str_'+str(i)] = globals()['path_polygon_'+str(a)+str(i)]
    globals()['aoi_geom_'+str(i)] = wkt.loads(globals()['wkt_str_'+str(i)])
    globals()['aoi_coords_'+str(i)] = list(globals()['aoi_geom_'+str(i)].coords)
    globals()['aoi_coords_'+str(i)] = [(y,x) for x,y in globals()['aoi_coords_'+str(i)]] 
    globals()['aoi_centroid'+str(i)]= globals()['aoi_geom_'+str(i)].centroid
    
    color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])]

    folium.PolyLine(globals()['aoi_coords_'+str(i)],
                                     color=color[0],
                                     #fill_color=color[0],
                                     fill_opacity=0.2,
                                     weight=3).add_to(m)
   

m
